# Code for behavioral means and figure S1
Uses dataframe obtained from epoched data together with metadata 

In [ ]:
import pandas as pd
import seaborn as sns
import feather
import os
import sys
import platform

if platform.system()=='Darwin':
    sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')



elif platform.system()=='Windows':
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    sys.path.append('E:/BBC/WP1/data/Code/python/BBC')

  
import matplotlib.pyplot as plt


In [ ]:
%load_ext autoreload


%autoreload 2

plt.style.use('default')
import matplotlib as mpl
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [ ]:
path='tsk/ana/behavioral/epoch_df'
filename='/tsk_n_epo_vep_cfa_df_P1_150.feather'

In [ ]:
df=pd.read_feather(path+filename)

In [ ]:
df

In [ ]:
df=df[df['difficulty']== 'normal']
df=df[df['accuracy']== 'correct']
df=df[df['sys_mask']== 1]

In [ ]:
39*10**-10


## Create DF which filters out 2.5 centiles from each side of the distribution


In [ ]:
list_gn=list()
for g_n in df['g_num'].unique():
    g_df=df[df['g_num']==g_n]
    g_df = g_df[(g_df.RT < g_df.RT.quantile(0.975))
                & (g_df.RT > g_df.RT.quantile(0.025))]

    list_gn.append(g_df)
df_filt=pd.concat(list_gn)

In [ ]:
df_filt

In [ ]:
filenameout='/tsk_n_epo_vep_cfa_df_P1_150_2_5quantile_symm_filtered.feather'
fileout=path+filenameout
feather.write_dataframe(df_filt,fileout)

In [ ]:
df_filt

In [ ]:
df=df_filt

### Anovas  on RT
anovas were subtited with GLMM see r file behavioral_GLM.Rmd

In [ ]:
df_group_rsp=df.pivot_table(index=['g_num','rsp_phase','awareness'],values='RT',aggfunc='mean')
df_group_rsp.reset_index(inplace=True)
df_group_card=df.pivot_table(index=['g_num','cardiac_phase','awareness'],values='RT',aggfunc='mean')
df_group_card.reset_index(inplace=True)

In [ ]:
import pingouin as pg

In [ ]:
pg.rm_anova(df_group_rsp,dv='RT',within=['awareness','rsp_phase'],subject='g_num',effsize='np2')

In [ ]:
pg.rm_anova(df_group_card,dv='RT',within=['awareness','cardiac_phase'],subject='g_num',effsize='np2')

In [ ]:
df_group_awa=df.pivot_table(index=['g_num'],columns=['awareness'],values='RT',aggfunc='mean')
df_group_awa.reset_index(inplace=True)



In [ ]:
print(f'mean aware = {df_group_awa["aware"].mean()} \nSD aware = {df_group_awa["aware"].std()} \nmean unaware = {df_group_awa["unaware"].mean()} \nSD unaware = {df_group_awa["unaware"].std()}')

## Figure S1 
Code to produce figure S1
phy_phase='cardiac_phase' produces figure for cardiac phase
phy_phase='rsp_phase' produces figure for respiratory phase

In [ ]:
dv='RT'
phy_phase='cardiac_phase'
dict_awa_list={}

if phy_phase=='rsp_phase':
    pivot_tab=df_group_rsp.copy()
else:
    pivot_tab=df_group_card.copy()

for awa in pivot_tab['awareness'].unique():


    awa_df=pivot_tab[pivot_tab['awareness']==awa][[dv,phy_phase]]
    for card in pivot_tab[phy_phase].unique():
        awa_card=awa_df[awa_df[phy_phase]==card]
        mean_card_awa=awa_card.mean()[dv]
        sem_card_awa=awa_card.sem()[dv]
        dict_awa_list[f'{awa}_{card}']={'mean':mean_card_awa,'sem':sem_card_awa}

In [ ]:
%matplotlib qt
ylab='RT'
if phy_phase=='rsp_phase':
    color_list=['g','tab:pink','g','tab:pink']
    facecolor_list=['w','w','g','tab:pink']
    xticklab=['Aware', 'Unaware']
    xlab='Awareness'
    cond_list=['aware_inh','aware_exh','unaware_inh','unaware_exh']
else:
    color_list=['tab:orange','b','tab:orange','b']
    facecolor_list=['w','w','tab:orange','b']
    xticklab=['Aware', 'Unaware']
    xlab='Awareness'
    cond_list=['aware_sys','aware_dia','unaware_sys','unaware_dia']


hatch_list=['///','///','','']
adj_list=[0,-0.4,-0.45,-0.85]
#color_list=['tab:orange','tab:orange','b','b']
#facecolor_list=['w','tab:orange','w','b']

alpha_list=[1,0.6,1,0.6]
x=0
fig,ax=plt.subplots(figsize=(9,7))
for cond,color,hatch,adj,facecolor,alpha in zip(cond_list,color_list,hatch_list,adj_list,facecolor_list,alpha_list):


    ax.bar(x=x+adj,height=[dict_awa_list[cond]['mean']],facecolor=facecolor,yerr=[dict_awa_list[cond]['sem']],
           edgecolor=color,hatch=hatch,linewidth=2.5,capsize=(2),ecolor='0.3',
           width=0.5,label=cond.split('_')[0]+' '+cond.split('_')[1],alpha=alpha)
    x+=1



plt.xticks([0.3,1.85],labels=xticklab,size='x-large')
plt.xlabel(xlab,size='x-large')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylabel(ylab,size='x-large')
#plt.title('Interaction effect of cadiac and awaraness condition in the P1')
plt.legend(bbox_to_anchor=(1,1),fontsize='x-large')

fig=plt.gcf()

In [ ]:
png=f'RT_int_{phy_phase}Xawa_2_5q.svg'
fig_path=f'ana/results_report/images/barplots/{png}'

fig.savefig(fig_path,dpi=1200,bbox_inches='tight',format='svg')

#accuracy and sys/dia ratio were produced on the behavioral df 


In [9]:
path='ana/behavioral'
filename='/behav_df_n.feather'
fileout='/behav_group.feather'
df=pd.read_feather(path+filename)

In [10]:
df

TF           RT difficulty accuracy awareness cardiac_phase  \
0        12553   609.187500     normal  correct     aware           dia   
1        14686   517.390625       easy  correct     aware           dia   
2        17366  1233.210938     normal  correct   unaware           dia   
3        19917  1026.179688     normal  mistake   unaware           sys   
4        25319   592.585938       easy  correct     aware           sys   
...        ...          ...        ...      ...       ...           ...   
25507  4363496  1158.992188     normal  correct   unaware           dia   
25508  4366363   800.593750     normal  correct     aware           dia   
25509  4369094   874.812500     normal  correct   unaware           sys   
25510  4371159   933.406250     normal  correct   unaware           sys   
25511  4373949   800.593750     normal  correct   unaware           dia   

            RRI  HeartRate  HeartRate_post  HeartRate_rolling_5_before  ...  \
0      0.927734  64.673684       64.335079                   62.615465  ...   
1      0.868164  69.111361       67.964602                   64.877039  ...   
2      0.986328  60.831683       58.181818                   65.410570  ...   
3      1.038086  57.798683       58.403042                   60.301070  ...   
4      1.028320  58.347578       62.186235                   58.548540  ...   
...         ...        ...             ...                         ...  ...   
25507  0.706055  84.979253       85.690377                   82.822591  ...   
25508  0.757812  79.175258       80.629921                   82.922684  ...   
25509  0.764648  78.467433       82.580645                   79.843252  ...   
25510  0.706055  84.979253       81.269841                   82.374446  ...   
25511  0.719727  83.364993       80.418848                   83.049583  ...   

         BPQ   Code  Sexe:    Age     mod_Z  log_norm    Zscore  rsp_int_inh  \
0      109.0   MQ50       M  24.0 -0.395391  6.412126 -0.453900     5.219727   
1      109.0   MQ50       M  24.0 -0.832647  6.248798 -0.872244     5.219727   
2      109.0   MQ50       M  24.0  2.577020  7.117377  2.389948     3.906250   
3      109.0   MQ50       M  24.0  1.590868  6.933598  1.446449     3.906250   
4      109.0   MQ50       M  24.0 -0.474469  6.384496 -0.529558     4.265625   
...      ...    ...     ...   ...       ...       ...       ...          ...   
25507   62.0  MQ108       F  26.0  2.141631  7.055306  0.919909     3.587891   
25508   62.0  MQ108       F  26.0  0.334784  6.685354 -0.147616     3.673828   
25509   62.0  MQ108       F  26.0  0.708954  6.774010  0.073452     3.673828   
25510   62.0  MQ108       F  26.0  1.004351  6.838841  0.247979     3.115234   
25511   62.0  MQ108       F  26.0  0.334784  6.685354 -0.147616     2.922852   

        inh_dur  mean_int  
0      2.950195  0.489818  
1      2.950195  0.489818  
2      1.833984  0.489818  
3      1.833984  0.489818  
4      1.921875  0.489818  
...         ...       ...  
25507  1.655273  0.017568  
25508  1.654297  0.017568  
25509  1.654297  0.017568  
25510  1.187500  0.017568  
25511  1.338867  0.017568  

[25512 rows x 63 columns]

In [15]:
col_dv_group=[ 'CACU_corr', 'corr']

In [16]:
df=df[df['sys_mask']==1]
df=df[df['difficulty']=='normal']

In [17]:
pivot_gen=df.pivot_table(index=['g_num'],values=col_dv_group)
pivot_gen.reset_index(inplace=True)
pivot_gen

g_num  CACU_corr  corr
0    g01       0.53  0.83
1    g03       0.63  0.72
2    g08       0.70  0.90
3    g10       0.70  0.91
4    g11       0.51  0.87
5    g12       0.50  0.92
6    g15       0.46  0.80
7    g16       0.49  0.77
8    g17       0.41  0.94
9    g19       0.63  0.84
10   g23       0.65  0.81
11   g24       0.34  0.84
12   g25       0.69  0.89
13   g28       0.33  0.74
14   g32       0.54  0.84
15   g33       0.47  0.95
16   g34       0.64  0.79
17   g37       0.43  0.88
18   g38       0.44  0.87
19   g39       0.33  0.89
20   g40       0.39  0.83
21   g41       0.37  0.96
22   g42       0.47  0.86
23   g44       0.38  1.00
24   g45       0.60  0.81
25   g46       0.63  0.87
26   g47       0.56  0.87
27   g49       0.47  0.78
28   g51       0.46  0.86
29   g52       0.39  0.82

In [18]:
print(pivot_gen['corr'].mean())
print(pivot_gen['corr'].std())
print(pivot_gen['CACU_corr'].mean())
print(pivot_gen['CACU_corr'].std())

0.8553333333333334
0.06441960150040718
0.5046666666666667
0.11536249714325031
